In [1]:
import pandas as pd
import numpy as np
import datetime

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Small data
data = pd.read_stata("../data/20170213 Cure complete tx-Stata 13.dta")

# 182 rows × 275 columns
data.drop_duplicates()

In [3]:
# Goal here is to find patients that do not have a followup/death date / are duplicates
all_182_ids = data["id"]

# This sheet has updated follow ups that were updated in 2020
dates_2020 = pd.read_csv("../data/2020_followup.csv")
dates_2020

# Drop duplicate follow ups, keeping the latest one
dates_2020[dates_2020["Study ID"].isin(["BT160", "DP051", "DS165", "KB017", "MR117"])]
dates_2020 = dates_2020.drop(dates_2020.index[[8, 12, 13, 28, 60]])
dates_2020 = dates_2020.drop_duplicates()
dates_2020["Last follow up date ( as in 2020)"] = dates_2020["Last follow up date ( as in 2020)"].fillna(dates_2020["Date of Death/Last seen in IPMS"])
dates_2020

# dates_2020 misses 39 patients. This sheet contains those 39 patients without updates
# Adjusted SM181 death/followup date based on original data
remaining_39 = pd.read_csv("../data/extra_2020_followup.csv")

# Combine data to have updated death dates/followups for all subjects
combined_182_fp_updates = pd.concat([dates_2020, remaining_39], ignore_index = True)

# Grab rows of interest
combined_182_fp_updates = combined_182_fp_updates[["Study ID", "Vital Status", "Last follow up date ( as in 2020)"]]
combined_182_fp_updates.rename(columns = {"Study ID":"id", "Vital Status":"vital_status", "Last follow up date ( as in 2020)":"death_followup"}, inplace = True)
combined_182_fp_updates["death_followup"] = pd.to_datetime(combined_182_fp_updates["death_followup"])

# Update in data
data = pd.merge(data, combined_182_fp_updates, on = "id").drop_duplicates()

In [4]:
len(set(data["id"]))

182

In [5]:
# Reseting factor columns to match large data

set(data["xmarital"]) # {'divorced/widowed', 'married/partnered', nan, 'single'}
data.loc[data["xmarital"] == 'divorced/widowed', 'xmarital'] = 2
data.loc[data["xmarital"] == 'single', 'xmarital'] = 3
data.loc[data["xmarital"] == 'married/partnered', 'xmarital'] = 1

data.loc[data["hiv_status"] == "positive", "hiv_status"] = 1
data.loc[data["hiv_status"] == "negative", "hiv_status"] = 0
data.loc[data["hiv_status"] == "not tested", "hiv_status"] = np.nan

data.loc[data["tumorresponse"] == "complete", "tumorresponse"] = 1
data.loc[data["tumorresponse"] == "partial", "tumorresponse"] = 2
data.loc[data["tumorresponse"] == "no_tx", "tumorresponse"] = 4
data.loc[data["tumorresponse"] == "to be assessed on followup", "tumorresponse"] = 3
data.loc[data["tumorresponse"] == "not recorded", "tumorresponse"] = 5

In [ ]:
# For patients who did not get eqd2 (radiation therapy), use pathology_date for treat_start_date (later, this will be used to calculate time_alive)
data["date_histology"] = pd.to_datetime(data["date_histology"], unit = 'D', origin = pd.Timestamp('1960-01-01'))
data.loc[data["date_radiation_begins"] == ".", "date_radiation_begins"] = np.nan
data["date_radiation_begins"] = pd.to_datetime(data["date_radiation_begins"])
data["date_radiation_begins"] = data["date_radiation_begins"].fillna(data["date_histology"])
data[data["date_radiation_begins"].isnull()] # One person without radiation start date

In [7]:
# Add cd4 closest to radiation start date

cd4 = data[data["hiv_status"] == 1][["id", "date_radiation_begins", "date_histology", "cd4_1", "date_1_cd4", "cd4_2", "date_2_cd4", "cd4_3", "date_3_cd4",
               "cd4_4", "date_4_cd4", "cd4_5", "date_5_cd4", "cd4_6", "date_6_cd4", "cd4_7", "date_7_cd4",
               "cd4_8", "date_8_cd4", "cd4_9", "date_9_cd4", "cd4_10", "date_10_cd4", "cd4_11", "date_11_cd4",
               "cd4_12", "date_12_cd4", "cd4_13", "date_13_cd4", "cd4_14", "date_14_cd4", "cd4_15", "date_15_cd4",
               "cd4_16", "date_16_cd4"]]
cd4.rename(columns = {"date_1_cd4":"date_1", "date_2_cd4":"date_2", "date_3_cd4":"date_3", "date_4_cd4":"date_4",
                      "date_5_cd4":"date_5", "date_6_cd4":"date_6", "date_7_cd4":"date_7", "date_8_cd4":"date_8",
                      "date_9_cd4":"date_9", "date_10_cd4":"date_10", "date_11_cd4":"date_11", "date_12_cd4":"date_12",
                      "date_13_cd4":"date_13", "date_14_cd4":"date_14", "date_15_cd4":"date_15", "date_16_cd4":"date_16"}, inplace = True)

# Pivot to grab everyone cd4 value that is closest to the radiation start date
cd4 = pd.wide_to_long(cd4, ["date_", "cd4_"], i = ["id", "date_radiation_begins", "date_histology"], j = "key").dropna(how = "all").reset_index()
# Create column to determine which cd4 date is closest to the radiation start date
cd4["date_radiation_begins"] = cd4["date_radiation_begins"].fillna(cd4["date_histology"])
cd4["date_difference"] = abs((cd4["date_radiation_begins"] - cd4["date_"]).dt.days)
# Sort by date difference column to grab first cd4_count
cd4 = cd4.sort_values(by = ["id", "date_difference"])
# 60/120 with CD4
cd4_to_merge = cd4.groupby("id")[["cd4_", "date_"]].first().reset_index().rename(columns = {"cd4_":"cd4_final"})
#print(cd4_to_merge.shape)
# Dropping those who do not have cd4 count (None dropped)
cd4_to_merge = cd4_to_merge.dropna(subset = ["cd4_final"])
# Dropping those who have default value (None dropped)
cd4_to_merge = cd4_to_merge[(cd4_to_merge["cd4_final"] != 9999) & (cd4_to_merge["cd4_final"] != 8888)]

# Add to data
data = data.merge(cd4_to_merge, how = "left", on = "id")


In [8]:
# Excluding patient because no radiation start date or pathology date
data = data[data["id"] != "LM188"]

In [9]:
# Add vl closest to radiation start date

vl = data[data["hiv_status"] == 1][["id", "date_radiation_begins", "date_histology", "VL_1", "date_1_vl", "vl_2", "date_2_vl", "vl_3", "date_3_vl",
               "vl_4", "date_4_vl", "vl_5", "date_5_vl", "vl_6", "date_6_vl", "vl_7", "date_7_vl",
               "vl_8", "date_8_vl", "vl_9", "date_9_vl", "vl_10", "date_10_vl", "vl_11", "date_11_vl",
               "vl_12", "date_12_vl", "vl_13", "date_13_vl", "vl_14", "date_14_vl", "vl_15", "date_15_vl", "vl_16",
               "date_16_vl"]]
vl.rename(columns = {"date_1_vl":"date_1", "date_2_vl":"date_2", "date_3_vl":"date_3", "date_4_vl":"date_4",
                      "date_5_vl":"date_5", "date_6_vl":"date_6", "date_7_vl":"date_7", "date_8_vl":"date_8",
                      "date_9_vl":"date_9", "date_10_vl":"date_10", "date_11_vl":"date_11", "date_12_vl":"date_12",
                      "date_13_vl":"date_13", "date_14_vl":"date_14", "date_15_vl":"date_15", "date_16_vl":"date_16"}, inplace = True)

# Pivot to grab everyone vl value that is closest to the radiation start date
vl = pd.wide_to_long(vl, ["date_", "vl_"], i = ["id", "date_radiation_begins", "date_histology"], j = "key").dropna(how = "all").reset_index()
# Create column to determine which vl date is closest to the radiation start date
vl["date_radiation_begins"] = vl["date_radiation_begins"].fillna(vl["date_histology"])
vl["date_difference"] = abs((vl["date_radiation_begins"] - vl["date_"]).dt.days)
# Sort by date difference column to grab first vl_count
vl = vl.sort_values(by = ["id", "date_difference"])
# 119/120 with vl
vl_to_merge = vl.groupby("id")[["vl_", "date_"]].first().reset_index().rename(columns = {"vl_":"vl_final"})
#print(vl_to_merge.shape)
# Dropping those who do not have vl count (None dropped)
vl_to_merge = vl_to_merge.dropna(subset = ["vl_final"])
# Dropping those who have default value (None dropped)
vl_to_merge = vl_to_merge[(vl_to_merge["vl_final"] != 9999) & (vl_to_merge["vl_final"] != 8888)]

# Add to data
data = data.merge(vl_to_merge, how = "left", on = "id")


In [11]:
data["surgery"] = 0

In [12]:
# Grab columns used in analysis
# Added notes:
## No final cancer stage captured in this data... Not needed for analysis
### Grabbing initial cancer stage and renaming to match combined cancer stage in large data
data = data[["id", "hiv_status", "age", "xmarital", "distance", "_2_have_you_been_screened_for_c",
                               "stage", "xstage", "vaginal_hemorrhage", "vaginal_discharge", "pelvic_pain",
                               "cr_1st", "hb_1st", "neut_1st", "wbc_1st", "performance_baseline", "cd4_final", "vl_final",
                               "on_arv", "No_chemo_cycles", "Total_received_ebrtfinal", "Total_received_Brachyfinal",
                               "EQD2Dosefinal", "Total_received_boostfinal", "tx_duration", "tumorresponse", 
                               "vital_status", "death_followup", "date_radiation_begins", "combivir", "neviripine",
                               "efavirenz", "truvada", "aluvia", "abacavir", "lamivuidine", "Stavudine", "on_tdf",
                               "cr_max_during_chemo", "hb_min_during_chemo", "wbc_min_during_chemo", "neut_min_during_chemo", "xmax_pelvic_pain", 
                               "xmax_vaginal_discharge", "xmax_fatigue", "xmax_weight_loss", 
                               "xmax_nausea", "xmax_vomiting", "xmax_urinary_frequency", "xmax_urinary_incontinence",
                               "xmax_urinary_urgency", "xmax_diarrhoea", "xmax_dermatitis_radiation", "surgery", "date_histology"]]

# Rename columns to match large data
# Not renaming if they match already: hiv_status, distance, vaginal_hemorrhage, vaginal_discharge,
# pelvic_pain, cd4_final, vl_final, on_arv, vital_status, combivir, efavirenz, truvada, aluvia,
# abacavir, surgery

data.rename(columns = {"id":"patient_id", "age":"enroll_age", "xmarital":"marital", "_2_have_you_been_screened_for_c":"cancer_screening",
                                "stage":"init_cancer_stage_exact", "xstage":"combined_cancer_stage", "cr_1st":"cr_result", "hb_1st":"hb_result", 
                                "neut_1st":"neut_result", "wbc_1st":"wbc_result", "performance_baseline":"init_performance_status",
                                "No_chemo_cycles":"total_chemo_received", "Total_received_ebrtfinal":"ebrt_curr_dose", 
                                "Total_received_Brachyfinal":"brachy_curr_dose", "EQD2Dosefinal":"eqd2", 'Total_received_boostfinal':'ebrt_boost_curr_dose',
                                "tx_duration":"treat_duration", "tumorresponse":"treat_response", "death_followup":"death_date", 
                                "date_radiation_begins":"treat_start_date", "neviripine":"nevirapine", "lamivuidine":"lamivudine",
                                "Stavudine":"stavudine", "on_tdf":"tdf", "cr_max_during_chemo":"cr_result_tox", "hb_min_during_chemo":"hb_min_result_tox", 
                                "wbc_min_during_chemo":"wbc_min_result_tox", "neut_min_during_chemo":"neut_min_result_tox",
                                "xmax_pelvic_pain":"pelvic_pain_tox", "xmax_vaginal_discharge":"vaginal_discharge_tox", "xmax_fatigue":"fatigue_tox",
                                "xmax_weight_loss":"weight_loss_tox", "xmax_nausea":"nausea_tox", "xmax_vomiting":"vomiting_tox", 
                                "xmax_urinary_frequency":"urine_freq_tox", "xmax_urinary_incontinence":"urine_incontinence_tox",
                                "xmax_urinary_urgency":"urine_urge_tox", "xmax_diarrhoea":"diarrhea_tox", "xmax_dermatitis_radiation":"dermatitis_tox",
                                "date_histology":"pathology_date"}, inplace = True)

# Transforming to match units of large data 
# Ex. 50 --> 5,000
data["ebrt_curr_dose"] = data["ebrt_curr_dose"]*100
# Ex. 50 --> 5,000
data["brachy_curr_dose"] = data["brachy_curr_dose"]*100

# Transforming factored variable to binary
# Alive --> 0, Dead --> 1
data.loc[(data["vital_status"] == "Alive") |
         (data["vital_status"] == "Alive "), "vital_status"] = 0
data.loc[data["vital_status"] == "Dead", "vital_status"] = 1

# In this data, combined_cancer_stage_exact is the same as init_cancer_stage_exact
data["combined_cancer_stage_exact"] = data["init_cancer_stage_exact"]

In [13]:
data.to_csv("small_data.csv", index = False)

# End